# Data Parallelism

![DataParallel](./pictures/DataParallel.png)

1) Вызывается `mindstore.communication.init` для инициализации коммуникационных ресурсов
2) Данные делятся на батчи (шарды), каждый шард имеет две переменные:`num_shards` и `shard_id`. По этим параметрам и происходят все операции по разделению датасета.
3) Изначально на всех частях инициализируются одинаковые значения. Это нужно для того, чтобы обеспечить синхронное обучение. Чтобы транслировать значения весов на разные ноды необходимо включить  parameter_broadcast
4) После этого со всех узлов собираются градиенты и происходит их агрегация. Для этого существует оператор `AllReduce`. У него есть функция mean. При ее включении происходит усреднение значений, однако существенно понижается производитльность
5) Происходит обновление параметров, из-за того, что параметр градиента усреднен и при этом начальные значения тоже одинаковы, то обновленные значения будут тоже одинаковы.  Если в сети задействована операция reduce над выборками, вывод сети может отличаться. Это определяется атрибутом сегментирования параллелизма данных.

# Semi-Automatic Parallelism

### Distributed operators and tensor distribution models
У MindSpore есть распределенные операторы и она умеет работать с распределенными тензорами. Пользователю о том, как это работает внутри думать не приходится. При этом MindSpore имеет 3 стратегии для распределения (как я понял, MindSPore может для каждого случая автоматически подбирать подходящую, хотя можно и задать самому). Их можно рассмотреть на примере умножения Тензора input на Тензор weight. 

1) **data parallelism**. При этой стратегии проиходит разрез input, weights не разрезается, задается так: `strategy=((2^N, 1, 1),(1, 1, 1))`*
2) **model parallelism**. При этой стратегии наоборот происходит разрез weights. Задается так: `strategy=((1, 1, 1),(2^N, 1, 1))`*
3) **mixed parallelism**. Разрезается и то, и другое. Задается так: `strategy=((2^N, 1, 1),(1, 1, 2^N))`*

На основе стратегии шардов в распределенном операторе определяется метод модели распределения входного и выходного тензора оператора. Распределенная модель состоит из      `device_matrix` (то, как происходит распределение), `tensor_shape` (рахмерность тензора), `tensor_map`(отношение между размерностями устройства и тензора). Распределенный оператор далее определяет, следует ли вставлять в граф дополнительные вычислительные и коммуникационные операции в соответствии с моделью тензорного распределения, чтобы обеспечить корректность логики работы оператора. (Суть этого предложение можно понять, если прочитать следующий пункт)

\* **Замечание:** 2^N, ибо MindSpore исходит из 2-х принципов: principle of base-2 and uniform distribution. Кортежах под цифрами обозначается, что можно делить и на сколько.

СДелать код

![tensor_parallel](./pictures/Tensor_parallel.png)

### Tensor distribution Transformation

На разных этапах могут применяться разные стратегии разделения операторов и тензопров. Поэтому необходимо как-то согласовывать выходные данные одного оператора с входными данными другого. MindStore также умеет это делать автоматически. Для примера можно рассмотнеть операцию: `Z=(X*W)*V`. <br>

Предположим, что сначала были разделены именно input данные. Тогда для их сбори применяется оператор `AllGather`

![AllGather](./pictures/Sample1-Dictributed_Transformation.png)

В следующем примере, наоборот используется model parallel. Для сборки и разделения, соответственно, используется оператор `AlltoAll`

![alt_all](./pictures/Sample2-Distributed_Transformation.png)

В последнем примере у нас между операциями с размерностями все хорошо. Однако затем необходимо уменьшить размерность. Для этого применяется оператор `AllReduce`

![AllReduce](./pictures/Sample3-Distributed_Transformation.png)

Здесь про это написано более подробно: https://www.mindspore.cn/tutorials/experts/en/r2.0/parallel/sharding_propagation.html

### Distrubuted auto-differentiation

При традиционном ручном срезе модели необходимо уделять внимание прямому сетевому взаимодействию и параллельной работе обратного сетевого соединения. MindSpore автоматически генерирует оператор обратной связи, инкапсулируя операцию связи в качестве оператора и используя исходную операцию автоматической дифференцировки фреймворка, поэтому даже при распределенном обучении пользователю нужно сосредоточиться только на прямом распространении сети, что действительно реализует полностью автоматический параллелизм обучения.

### Support multi-dimensional hybrid parallelism

MindSpore поддерживает несколько режимов параллельной работы, которые можно использовать вместе

**operator-level parallelism**

Параллелизм операторов принимает операторы в нейронной сети и разделяет входной тензор на несколько устройств для вычислений. Таким образом, выборки данных и параметры модели могут быть распределены между различными устройствами для обучения крупномасштабных моделей глубокого обучения и использовать ресурсы кластера для параллельных вычислений для повышения общей скорости. Стоит отметить, что MindSPore с таким видом параллелизма может работать автоматически

Код для использования на MPI: https://gitee.com/mindspore/docs/tree/r2.3.1/docs/sample_code/distributed_operator_parallel <br>
Объяснение кода: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/operator_parallel.html#basic-principle


# Изучить подробно

**piepline parallelism**

При большом количестве устройств кластера, если используется только **operator parallelism**, требуется обмен данными через область обмена данными всего кластера, что может сделать обмен данными неэффективным и, следовательно, снизить общую производительность. Конвейерный параллелизм может разделить структуру нейронной сети на несколько этапов, и каждый этап выполняется в определенной части устройства, что ограничивает область коллективного взаимодействия этой частью устройства, в то время как на промежуточном этапе используется двухточечная связь. 

**Базовый принцип**. Параллельный конвейер — это разделение операторов в нейронной сети на несколько этапов, а затем сопоставление этапов с разными устройствами, чтобы разные устройства могли вычислять разные части нейронной сети. Сеть из 4 слоев MatMul разбивается на 4 этапа и распределяется по 4 устройствам. При прямых вычислениях каждая машина отправляет результат следующей машине через оператора связи после вычисления MatMul на машине, и в это же время следующая машина получает (Receive) результат MatMul предыдущей машины через оператора связи, и начинает рассчитывать MatMul на машине; При обратном расчете, после того как градиент последней машины вычислен, результат отправляется предыдущей машине, и в то же время предыдущая машина получает результат градиента последней машины и начинает вычислять обратный результат текущей машины.

![piepline_parallelism](./pictures/Piepline_parallelism-basic_principe.png)

Простое разделение модели на несколько устройств не приведет к повышению производительности, поскольку в линейной структуре модели одновременно работает только одно устройство, в то время как другие устройства ожидают, что приводит к пустой трате ресурсов. Чтобы повысить эффективность, параллельный конвейер дополнительно делит малую партию (MiniBatch) на более мелкие микропакеты (MicroBatch) и принимает последовательность выполнения конвейера в микропакете, чтобы достичь цели повышения эффективности

**GPipe screduler**.Малые партии разрезаются на 4 микропакета, а 4 микропакета выполняются на 4 группы, образуя конвейер. Для обновления параметров используется градиентная агрегация микропартии, где каждое устройство только хранит и обновляет параметры соответствующей группы. где порядковый номер белого цвета представляет индекс микропартии.

![GPipe](./pictures/Pipeline_parallelism-GPipe.png)

**1F1B screduler**.В параллельной реализации конвейера MindSpore порядок выполнения был скорректирован для лучшего управления памятью. Обратный микропакет с номером 0 выполняется сразу после его прямого выполнения, так что память промежуточного результата микропакета с номером 0 освобождается раньше, тем самым гарантируя, что пиковое использование памяти будет ниже

![1F1B](./pictures/Pipeline_parallelism-1F1B.png)

**Interleaved Pipeline Scheduler**.Чтобы повысить эффективность параллелизма конвейеров и уменьшить долю простоев, компания Megatron LM предлагает новую систему параллельного планирования конвейеров под названием "чередующийся конвейер". Традиционный конвейерный параллелизм обычно предусматривает размещение нескольких последовательных слоев модели (например, слоев трансформера) на одной рабочей площадке. При планировании конвейера с чередованием на каждом этапе выполняются вычисления с чередованием на неперерывных слоях модели, чтобы еще больше уменьшить долю простоев с большей связью. Например, при традиционном конвейерном параллелизме каждая стадия имеет 2 слоя модели, а именно: стадия 0 имеет слои 0 и 1, стадия 1 имеет слои 2 и 3, стадия 3 имеет слои 4 и 5, а стадия 4 имеет слои 6 и 7, в то время как в конвейере с чередованием на стадии 0 есть слои 0 и 4, стадия 1 содержит слои 1 и 5, стадия 2 содержит слои 2 и 6, а стадия 3 содержит слои 3 и 7.

![Intervalled+1F1B](./pictures/Pipeline_patallelism-Intervalled_and_1F1B.png)

**MindStore screduler**

![MindStore](./pictures/Pipeline_parallelism-MindStore_screduler.png)

Код: https://gitee.com/mindspore/docs/tree/r2.3.1/docs/sample_code/distributed_pipeline_parallel <br>
Объяснение кода: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/pipeline_parallel.html

**MoE parallelism** 
MoE заключается в распределении экспертов между разными работниками, и каждый работник получает разные пакеты обучающих данных. Для уровня, не связанного с MoE, параллелизм экспертов - это то же самое, что параллелизм данных. На уровне MoE токены в последовательности отправляются работникам, соответствующим их соответствующим экспертам, посредством связи "все ко всем". После завершения вычислений соответствующего эксперта они затем повторно передаются исходному работнику по принципу "все ко всем" и упорядочиваются в исходную последовательность для вычисления следующего уровня. Поскольку модели MoE обычно содержат большое количество экспертов, параллелизм экспертов увеличивается в большей степени с увеличением размера модели, чем параллелизм моделей.

![MoE](./pictures/MoE_parallelism.png)

**Multi-Copy parallelism**

Данные входной модели разрезаются в соответствии с размерностью batchsize, таким образом, существующая форма с одной копией преобразуется в форму с несколькими копиями, так что нижележащий слой взаимодействует, в то время как другая копия выполняет вычислительную операцию без ожидания. Это гарантирует, что время вычислений и обмена данными нескольких копий дополняет друг друга для повышения производительности модели, в то время как разделение данных на несколько копий также уменьшает количество параметров входных данных операторов, тем самым сокращая время вычислений отдельных операторов, что значительно помогает повысить производительность модели.

![Multi-Copy](./pictures/Multi-copy_parallelism.png)

**Пояснение:** Multi-copy parallelism — это метод параллельных вычислений, используемый для ускорения обучения моделей, когда одна и та же модель копируется и выполняется на нескольких устройствах (GPU, TPU, или узлах кластера) одновременно, с различными наборами данных.

**Замечание:** Как я понял реализации в MindStore этих двух видов параллелизма нет

**Optimizer parallelism**

При обучении параллелизму данных или параллелизму операторов одна и та же копия параметров модели может существовать на нескольких устройствах, что позволяет оптимизатору выполнять избыточные вычисления на нескольких устройствах при обновлении этого веса. В этом случае вычисления оптимизатора могут быть распределены по нескольким устройствам за счет параллелизма оптимизатора. Его преимущества заключаются в снижении потребления статической памяти и объема вычислений в оптимизаторе

**Замечание:** В режиме AUTO_PARALLEL или SEMI_AUTO_PARALLEL включается optimizer parallelism, если параметры после стратегии нарезки имеют повторяющиеся срезы между машинами, а максимальный размер фигуры делится на количество повторяющихся срезов, фреймворк сохраняет параметры как минимальные срезы и обновляет их в оптимизаторе. В этом режиме поддерживаются все оптимизаторы

**Что мы хотим сделать и зачем это надо?**

Традиционная параллельная модель данных хранит копии параметров модели на каждом устройстве, разрезает обучающие данные, синхронизирует информацию о градиенте после каждой итерации с помощью операторов связи и, наконец, обновляет параметры с помощью вычислений оптимизатора. Параллелизм данных, хотя и эффективен для повышения производительности обучения, не позволяет максимально эффективно использовать машинные ресурсы. Оптимизатор вводит избыточную память и вычисления, устранение этих избыточностей является точкой оптимизации, на которой следует сосредоточиться.

В обучающей итерации параллелизм данных вводит операцию связи для синхронизации градиентов по нескольким карточкам для сбора градиентов параметров, созданных различными выборками на каждой карте. Поскольку параллелизм модели не используется, операции оптимизатора на каждой карте фактически обновляются на основе одних и тех же параметров и в том же направлении. Основная идея устранения избыточности оптимизатора заключается в том, чтобы распределить эту память и вычисления по картам для достижения прироста памяти и производительности.

**Inner-layer**. Одной из групп весов является межслойное деление параметров и градиентов внутри оптимизатора, а общий поток обучения показан на рисунке 1. Параметры и градиенты группируются на различных картах для обновления, а затем обновленные веса распространяются между устройствами с помощью операции коммуникационной трансляции. Прирост памяти и производительности решения зависит от группы с наибольшей долей параметров. Когда параметры разделены поровну, теоретические положительные выигрыши равны (N-1)/N времени выполнения оптимизатора и динамической памяти и (N-1)/N объема памяти для параметров состояния оптимизатора, где N обозначает количество устройств. А отрицательное преимущество — это время связи, которое наступает при совместном использовании весов сети.

![Inner-layer](./pictures/Optimizer_parallelism-inner-layer.png)

**Пояснение:** Как я понял, обратное распростанение тоже может выполнять как бы параллельно. Просто градиенты вычисляются не полностью, а как бы локально для части. Затем значения аггрегируются и уже идут оптимизатору

Еще один способ реализации нарезки параметров — это внутрислойное разделение параметров, и для каждого параметра берется соответствующий срез и градиент в соответствии с номером устройства. После обновления параметров и градиентов вызывается операция агрегации связи для совместного использования параметров между устройствами. Преимущество этой схемы в том, что она естественно поддерживает балансировку нагрузки, т.е. количество параметров и вычислений согласовано на каждой карте, а недостаток в том, что форма параметра должна быть кратна количеству устройств. Теоретические преимущества этой схемы согласуются с группировкой параметров, и в систему были внесены следующие усовершенствования с целью расширения преимуществ.

Во-первых, разделение весов в сети может еще больше уменьшить статическую память. Однако для этого также требуется выполнить операцию с общим весом в конце итерации перед прямым началом следующей итерации, гарантируя, что исходная форма тензора останется неизменной после перехода к прямым и обратным операциям. Кроме того, основным отрицательным выигрышем от параллельной работы оптимизатора является время связи общих весов, которое может принести выигрыш в производительности, если мы сможем его уменьшить или скрыть. Одним из преимуществ перекрестной итерации связи является то, что операции связи могут выполняться вперемежку с прямой сетью путем объединения операторов связи в соответствующие группы, тем самым максимально скрывая затраты времени на связь. Затраты времени на общение также связаны с объемом общения. Для сети со смешанной точностью, если мы сможем использовать связь fp16, объем связи уменьшится вдвое по сравнению с fp32.

![MindStore](./pictures/Optimizer_parallelism-MindStore.png)

**Пояснение:** Что такое перерестаня итерация?

**Перекрестная итерация** (cross-iteration) представляет собой технику, которая позволяет различным частям нейронной сети обмениваться информацией на протяжении процесса обучения. Это особенно полезно при работе с большими и сложными моделями, которые разбиваются на несколько частей для параллельной обработки.

Как работает перекрестная итерация?
1) Разбиение сети: Нейронная сеть делится на несколько более мелких подсетей.
2) Параллельная обработка: Каждая подсеть обрабатывает свою часть данных независимо.
3) Обмен информацией: После определенного числа итераций (или эпох) обучения, подсети обмениваются информацией о своих текущих весах или активациях.
4) Обновление весов: На основе полученной информации, каждая подсеть обновляет свои веса, чтобы лучше согласовать свою работу с другими подсетями.

Код: https://gitee.com/mindspore/docs/tree/r2.3.1/docs/sample_code/distributed_optimizer_parallel <br>
Объяснение кода: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/optimizer_parallel.html


# Изучить подробно

### Fully automatic parallelism
Полностью автоматический параллелизм основан на полуавтоматическом фреймворке MindSpore, заменяющем экспертную конфигурацию параллельных стратегий на автоматические гибридные алгоритмы генерации параллельных стратегий. Пользователи разрабатывают свои собственные модели нейронных сетей (или импорты MindIR) с использованием языка Python, которые разбираются в вычислительные графы (графы ANF) с помощью MindSpore. Модуль автоматической гибридной параллельной генерации стратегий ищет лучшую стратегию через алгоритм и передает ее полуавтоматическому параллельному модулю, который анализирует распределение тензоров, анализ распределенных операторов, управление устройствами, выполняет нарезку всего графа и передает его на серверную часть для вычислений.

![overview](./pictures/Fully_automatic_parallelism-overview.png)

По сути, модуль генерации гибридных параллельных стратегий отвечает за поиск подходящей стратегии параллельного сегмента для данной модели нейронной сети и конфигурации кластера. Ключевой используемой технологией является алгоритм поиска стратегии на основе cost model, которая строит cost model для описания computetion cost и communication cost в сценарии распределенного обучения и использует затраты памяти в качестве ограничения для эффективного поиска параллельной стратегии с лучшей производительностью с помощью алгоритма поиска вычислительного графа.

**Как он это делает?**

Полностью автоматический параллелизм очень сложен в реализации, и MindSpore делит предоставленный алгоритм генерации стратегии на уровень L1 и уровень L2 в зависимости от степени требуемого вмешательства пользователя (здесь мы предполагаем, что настроенная вручную стратегия полного графа SEMI_AUTO — это уровень L0, а схема, не требующая участия пользователя, — уровень L3).

**L1 (Sharding Propagation)**. В этом режиме пользователю нужно вручную определить стратегии для нескольких ключевых операторов, а стратегии для остальных операторов в вычислительном графе автоматически генерируются алгоритмом. Поскольку стратегия ключевого оператора определена, стоимостная модель алгоритма в основном описывает стоимость Tensor redistribution между операторами, а целью оптимизации является минимизация стоимости Tensor redistribution всего графа. Поскольку стратегия основного оператора была определена, что эквивалентно сжатому пространству поиска, время поиска этой схемы короче, и эффективность ее стратегии зависит от определения стратегии ключевого оператора. <br>
**Как это работает?**. Входные данные Sharding Propagation представляют собой вычислительный граф, в котором узлы представляют операторы, а ребра кодируют отношения зависимости данных операторов. Из определения модели с некоторыми операторами, настроенными для стратегий сегментирования, Sharding Propagation выполняется следующим образом:
1) Генерация возможных стратегий шардинга для ненастроенных операторов;
2) Генерация тензорных перераспределений и связанных с ними затрат на коммуникацию для каждого ребра;
3) Начиная с настроенных операторов стратегия сегментирования переходит на ненастроенные, используя BFS, с целью минимизации затрат на обмен данными вдоль каждого ребра.

![sheme](./pictures/Sharding_proparation.png)

Имея вычислительный граф с некоторыми настроенными стратегиями, сначала перечисляются возможные стратегии для ненастроенных операторов, как показано на рисунке (b). Далее в нем перечисляются возможные стратегии и затраты на перераспределение тензора для каждого ребра. Как показано на рисунке (c), стратегия для ребра определена в виде пары [s_strategy, t_strategy], где s_strategy и t_strategy обозначают стратегию шардинга для исходного и целевого операторов соответственно. Наконец, начиная с настроенного оператора, он определяет стратегию сегментирования следующего оператора, так что затраты на связь при тензорном перераспределении сведены к минимуму. Распространение заканчивается, когда стратегии сегментирования для всех операторов рассчитаны, как показано на рисунке (d).

ссылка: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/sharding_propagation.html

**L2**. Существует два типа алгоритмов генерации стратегий уровня L2: динамическое программирование и символьный автоматический параллельный планировщик (сокращенно SAPP). Оба способа имеют свои преимущества и недостатки. Алгоритм динамического программирования способен искать оптимальную стратегию, заложенную в стоимостную модель, но для поиска параллельных стратегий для огромных сетей требуется больше времени. Алгоритм SAPP способен мгновенно генерировать оптимальные стратегии для огромных сетей и крупномасштабных сокращений. Основная идея алгоритма динамического программирования состоит в том, чтобы построить стоимостную модель полного графа, включающую стоимость вычислений и стоимость связи, описать абсолютную временную задержку в распределенном процессе обучения и сжать время поиска с помощью эквивалентных методов, таких как устранение ребер и исключение точек, но на самом деле пространство поиска растет экспоненциально с увеличением количества устройств и операторов. Поэтому он не эффективен для больших кластеров с большими моделями. SAPP моделируется на основе принципа параллелизма путем создания абстрактной машины для описания топологии аппаратного кластера и оптимизации модели стоимости путем символьного упрощения. Его стоимостная модель сравнивает не прогнозируемую абсолютную задержку, а относительную стоимость различных параллельных стратегий, поэтому она может значительно сжать пространство поиска и гарантировать минимальное время поиска для кластеров из 100 карт.

### Гибридный параллелизм

В настоящее время гибридные параллельные вычисления обычно используются следующие сценарии: гибридность оптимизатора, гибридность встраивания и гибридность PS (Parameter Server).

**Optimizer Heterogenetity**

Во время обучения большой модели состояние оптимизатора занимает большой объем памяти, что, в свою очередь, ограничивает размер модели, которую можно обучить. Оптимизатор и все его вычисления перебрасываются на CPU.

![Optimizer_heterogenenity](./pictures/Optimizer_heterogenenity.png)

**Embendding Heterogenetity**

В некоторых сетях, где необходимо проверять большие таблицы внедрения, размер таблиц встраивания часто составляет сотни гигабайт, что ограничено объемом памяти ускорителя и не может быть выполнено путем загрузки всей таблицы непосредственно на ускоритель. Поместив операторов, подключенных к таблице весов на процессоре для выполнения, мы избегаем проблемы, связанной с тем, что ускоритель не может обучить сеть из-за ограниченности памяти.

![Embeddings_heterogenetity](./pictures/Embeddings_heterogenetity.png)

**PS Heterogenetity**

**Parameter Server** — широко используемая архитектура в распределенном обучении, которая состоит из трех независимых компонентов: сервера, воркера и планировщика. Их функции заключаются в следующем:

**Сервер:** сохраняет веса модели и градиенты обратного вычисления и обновляет модель с использованием градиентов, отправленных рабочими.

**Воркер:** выполняет прямые и обратные вычисления в сети. Значение градиента для обратного вычисления загружается на сервер через API`Push`, а обновленная сервером модель загружается для воркера через API `Pull`.

**Планировщик:** устанавливает отношения связи между сервером и воркером.

Реализация: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/parameter_server_training.html

![PS](./pictures/PS_heterogenetity.png)

Ссылка, где написано про параллелизм: https://www.mindspore.cn/docs/en/r2.3.1/design/distributed_training_design.html